In [ ]:
#beginning, Team6

In [1]:
%matplotlib inline
import matplotlib
import os              # This provides several system utilities
import pandas as pd    # This is the workhorse of data munging in Python
import numpy as np     # This is for general numerical operations 
import seaborn as sns  # This allows us to efficiently and beautifully plot
import networkx as nx  
import osmnx as ox
import statsmodels.api as sm
import matplotlib.pyplot as plt

# display settings (optional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [2]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname = infodir + r'\data\buurten_filtered.csv'

gpdf01 = pd.read_csv(fname)
#this is to verify
gpdf01.head()

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013


In [3]:
#Team6 is dropping all entries that do not have "doordeweeks avond" in "Dagdeel"
#Team6 chose to focus on the most likely time cars will be at home after a day at work
gpdf01_1 = gpdf01[gpdf01['Dagdeel'] == 'doordeweeks avond']
#Team6 is creating column Wijken, to merge more easily with future databases for maps
gpdf01_1['Wijken'] = gpdf01_1['BRT'].str[:2]
#team6 is creating a column with the buurt code to keep it
gpdf01_1['BRT_code'] = gpdf01_1['BRT'].astype(str).str[:4]
#team6 is separating the name of each neighborhood to merge more easily with future databases
gpdf01_1['BRT_name'] = gpdf01_1['BRT'].str[5:]
#team6 is replacing all dashes in the name of each neighborhood to merge more easily with future databases
gpdf01_1['BRT_name'] = gpdf01_1['BRT_name'].str.replace('-', ' ', regex=False)
gpdf01_1.head(10)

C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpdf01_1['Wijken'] = gpdf01_1['BRT'].str[:2]
C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpdf01_1['BRT_code'] = gpdf01_1['BRT'].astype(str).str[:4]
C:\Users\EPRO\AppData\Local\Temp\ipykernel_1984\2552515082.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Wijken,BRT_code,BRT_name
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023,AA,AA01,Planciusbuurt Noord
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401,AA,AA02,Westelijke eilanden
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417,AA,AA03,Westerdokseiland
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947,AA,AA04,Planciusbuurt Zuid
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013,AA,AA05,Haarlemmerbuurt West
5,122.0,AA06 Haarlemmerbuurt-Oost,A,202403-202404,doordeweeks avond,121.0,110.0,1.0,0.0,111.0,91.0,92.0,"POLYGON((4.8901799 52.3823939,4.8901504 52.382...","POLYGON((52.3823939 4.8901799,52.3823625 4.890...",4.892116,52.380579,AA,AA06,Haarlemmerbuurt Oost
6,123.0,AB01 Marnixbuurt-Noord,A,202403-202404,doordeweeks avond,169.0,137.0,1.0,0.0,138.0,81.0,82.0,"POLYGON((4.8829591 52.3836506,4.8829307 52.383...","POLYGON((52.3836506 4.8829591,52.3837005 4.882...",4.880560,52.380942,AB,AB01,Marnixbuurt Noord
7,124.0,AB02 Driehoekbuurt,A,202403-202404,doordeweeks avond,312.0,266.0,3.0,0.0,269.0,85.0,86.0,"POLYGON((4.8829591 52.3836506,4.8803612 52.379...","POLYGON((52.3836506 4.8829591,52.37958 4.88036...",4.884213,52.381615,AB,AB02,Driehoekbuurt
8,125.0,AB03 Anjeliersbuurt-Noord,A,202403-202404,doordeweeks avond,165.0,141.0,0.0,0.0,141.0,85.0,85.0,"POLYGON((4.8871229 52.3790942,4.8881601 52.380...","POLYGON((52.3790942 4.8871229,52.3802151 4.888...",4.883804,52.379197,AB,AB03,Anjeliersbuurt Noord
9,126.0,AB04 Zaagpoortbuurt,A,202403-202404,doordeweeks avond,158.0,85.0,2.0,0.0,87.0,54.0,55.0,"POLYGON((4.8791773 52.3777046,4.8786863 52.377...","POLYGON((52.3777046 4.8791773,52.3778162 4.878...",4.877343,52.376228,AB,AB04,Zaagpoortbuurt


In [4]:
#team6 is checking how many we have left
before = len(gpdf01)
after = len(gpdf01_1)
print(f"Rows before: {before}, after: {after}, dropped: {before - after}")

Rows before: 1447, after: 518, dropped: 929


In [5]:
#team6 is purging all rows with "netto_pkd" and "bruto_pkd" higher than 150% because that looks like missing and damaged data
#that would otherwise interfere with our next process
gpdf01_1[(gpdf01_1['Netto_PKD'] <= 150) & (gpdf01_1['Bruto_PKD'] <= 150)]
gpdf01_1.head(10)

,OBJECTNUMMER,BRT,SDC,Periode,Dagdeel,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD,WKT_LNG_LAT,WKT_LAT_LNG,LNG,LAT,Wijken,BRT_code,BRT_name
0,117.0,AA01 Planciusbuurt-Noord,A,202403-202404,doordeweeks avond,55.0,52.0,3.0,0.0,55.0,95.0,100.0,"POLYGON((4.8857454 52.3881665,4.8844789 52.388...","POLYGON((52.3881665 4.8857454,52.3883785 4.884...",4.884390,52.387023,AA,AA01,Planciusbuurt Noord
1,118.0,AA02 Westelijke eilanden,A,202403-202404,doordeweeks avond,313.0,287.0,9.0,0.0,296.0,92.0,95.0,"POLYGON((4.8910092 52.3884398,4.8859714 52.388...","POLYGON((52.3884398 4.8910092,52.3881287 4.885...",4.887548,52.385401,AA,AA02,Westelijke eilanden
2,119.0,AA03 Westerdokseiland,A,202403-202404,doordeweeks avond,165.0,76.0,0.0,0.0,76.0,46.0,46.0,"POLYGON((4.8950776 52.3886909,4.8923612 52.388...","POLYGON((52.3886909 4.8950776,52.3885233 4.892...",4.894474,52.384417,AA,AA03,Westerdokseiland
3,120.0,AA04 Planciusbuurt-Zuid,A,202403-202404,doordeweeks avond,20.0,15.0,1.0,0.0,16.0,75.0,80.0,"POLYGON((4.8839232 52.385505,4.884072 52.38566...","POLYGON((52.385505 4.8839232,52.3856666 4.8840...",4.882867,52.384947,AA,AA04,Planciusbuurt Zuid
4,121.0,AA05 Haarlemmerbuurt-West,A,202403-202404,doordeweeks avond,148.0,141.0,4.0,0.0,145.0,95.0,98.0,"POLYGON((4.8839232 52.385505,4.8835118 52.3849...","POLYGON((52.385505 4.8839232,52.3849538 4.8835...",4.886377,52.383013,AA,AA05,Haarlemmerbuurt West
5,122.0,AA06 Haarlemmerbuurt-Oost,A,202403-202404,doordeweeks avond,121.0,110.0,1.0,0.0,111.0,91.0,92.0,"POLYGON((4.8901799 52.3823939,4.8901504 52.382...","POLYGON((52.3823939 4.8901799,52.3823625 4.890...",4.892116,52.380579,AA,AA06,Haarlemmerbuurt Oost
6,123.0,AB01 Marnixbuurt-Noord,A,202403-202404,doordeweeks avond,169.0,137.0,1.0,0.0,138.0,81.0,82.0,"POLYGON((4.8829591 52.3836506,4.8829307 52.383...","POLYGON((52.3836506 4.8829591,52.3837005 4.882...",4.880560,52.380942,AB,AB01,Marnixbuurt Noord
7,124.0,AB02 Driehoekbuurt,A,202403-202404,doordeweeks avond,312.0,266.0,3.0,0.0,269.0,85.0,86.0,"POLYGON((4.8829591 52.3836506,4.8803612 52.379...","POLYGON((52.3836506 4.8829591,52.37958 4.88036...",4.884213,52.381615,AB,AB02,Driehoekbuurt
8,125.0,AB03 Anjeliersbuurt-Noord,A,202403-202404,doordeweeks avond,165.0,141.0,0.0,0.0,141.0,85.0,85.0,"POLYGON((4.8871229 52.3790942,4.8881601 52.380...","POLYGON((52.3790942 4.8871229,52.3802151 4.888...",4.883804,52.379197,AB,AB03,Anjeliersbuurt Noord
9,126.0,AB04 Zaagpoortbuurt,A,202403-202404,doordeweeks avond,158.0,85.0,2.0,0.0,87.0,54.0,55.0,"POLYGON((4.8791773 52.3777046,4.8786863 52.377...","POLYGON((52.3777046 4.8791773,52.3778162 4.878...",4.877343,52.376228,AB,AB04,Zaagpoortbuurt


In [6]:
#team6 slicing database to focus on the main usefull content
gpdf01_2 = gpdf01_1.drop(columns=['BRT', 'Dagdeel', 'WKT_LNG_LAT', 'WKT_LAT_LNG', 'LNG', 'LAT'])
#ordering the database columns to be easier to read
gpdf01_2 = gpdf01_2[['OBJECTNUMMER', 'Wijken', 'BRT_code', 'BRT_name'] + [col for col in gpdf01_2.columns if col not in ['OBJECTNUMMER', 'Wijken', 'BRT_code', 'BRT_name']]]
gpdf01_2.head()

,OBJECTNUMMER,Wijken,BRT_code,BRT_name,SDC,Periode,Cap_open,Bez_open,Fout,Object,Bez_tot,Netto_PKD,Bruto_PKD
0,117.0,AA,AA01,Planciusbuurt Noord,A,202403-202404,55.0,52.0,3.0,0.0,55.0,95.0,100.0
1,118.0,AA,AA02,Westelijke eilanden,A,202403-202404,313.0,287.0,9.0,0.0,296.0,92.0,95.0
2,119.0,AA,AA03,Westerdokseiland,A,202403-202404,165.0,76.0,0.0,0.0,76.0,46.0,46.0
3,120.0,AA,AA04,Planciusbuurt Zuid,A,202403-202404,20.0,15.0,1.0,0.0,16.0,75.0,80.0
4,121.0,AA,AA05,Haarlemmerbuurt West,A,202403-202404,148.0,141.0,4.0,0.0,145.0,95.0,98.0


In [7]:
#for the import process, next step is to create a string variable containing the directory of the file and a file name variable
infodir2 = r'C:\Users\EPRO\OneDrive - Wageningen University & Research\Course Channel Metropolitan Challenges 2025 - Paper Group 6 Parking space vs Food Space'
fname2 = infodir2 + r'\data\KEA_db_green_and_gray_ngbhd_2023_03_trnsltd_only_AMS.csv'

gpdf02g = pd.read_csv(fname2)
#this is to verify
gpdf02g.head()

,OBJID,mnpty_code,mnpty_name,ngbhd_code,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,qty_inhbt,perc_built_area,perc_water_total,m2_water,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,perc_publcnt,m2_publc_green,perc_public_green,perc_publc_tree,perc_trnsn_total,perc_publcnt_tree,perc_publcnt_green,perc_publcnt_greennt,perc_other,...,m2_built_envmt,m2_built_envmt_greennt,m2_green_other,m2_other_surfc,m2_other_greennt,m2_tree_publc,m2_tree_publcnt,m2_tree_built_envmt,m2_tree_water,perc_green,perc_low_green,perc_tree,perc_built_gray,m2_agrct_green,m2_agrct,m2_agrct_greennt,m2_trnsn_greennt,m2_green_trnsn,m2_trnsn,m2_agrct_tree,m2_tree_other,m2_tree_trnsn,perc_trnsn_tree,perc_trnsn_green,perc_trnsn_greennt
0,7,GM0363,Amsterdam,BU03631306,4427,Westergasfabriek,1,-99999999,0,99.999999,12.47,26504.03,255419.18,7.28,99685.39,68.27,10.31,44776.51,17.53,29.24,1.67,1.06,0.43,8.82,0.0,...,18382.65,18382.65,0.0,4.67,4.67,74683.12,2719.53,199.06,5333.44,50.53,18.01,32.52,39.09,0.0,0.0,0.0,4022.37,130.2,4152.57,0.0,0.0,130.49,0.23,3.28,3.60
1,45,GM0363,Amsterdam,BU03634709,4552,Duivelseiland,1,23729,1385,100.000003,4.97,2920.45,61646.98,34.69,23080.23,42.51,17.82,202.57,0.33,5.07,0.00,5.94,0.69,11.19,0.0,...,20876.45,20876.45,0.0,0.00,0.00,3126.23,3662.17,514.10,141.05,13.09,1.02,12.07,82.16,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.46,4.27,8.62
2,53,GM0363,Amsterdam,BU03634600,4539,Valeriusbuurt Oost,1,12248,1405,99.999996,4.55,4607.40,120651.50,32.67,38367.50,40.32,22.34,2521.47,2.09,8.52,0.12,8.73,1.51,12.10,0.0,...,38507.74,38507.74,0.0,0.00,0.00,10281.06,10538.49,906.86,879.44,22.33,3.60,18.73,73.85,0.0,0.0,0.0,144.76,0.0,144.76,0.0,0.0,0.00,1.25,1.37,1.83
3,90,GM0363,Amsterdam,BU03635301,4579,IJselbuurt Oost,1,21880,2900,99.999988,16.80,26146.56,159417.36,25.28,54516.32,40.28,17.55,2078.39,1.30,6.09,0.10,5.97,2.77,8.81,0.0,...,39400.79,39378.99,0.0,0.00,0.00,9703.89,9513.20,901.93,631.04,17.14,4.09,13.05,66.47,0.0,0.0,0.0,109.68,8.4,118.08,0.0,0.0,32.23,0.03,0.68,9.48
4,92,GM0363,Amsterdam,BU03639303,4774,F-buurt,1,20169,4465,100.000004,6.38,14141.01,233166.41,24.11,121287.55,56.32,13.12,12907.19,5.54,4.30,0.06,0.43,0.42,12.27,0.0,...,56080.41,56080.41,0.0,0.00,0.00,10036.16,1006.84,141.33,735.41,11.07,5.96,5.11,82.86,0.0,0.0,0.0,145.69,0.0,145.69,0.0,0.0,0.00,0.00,0.01,16.96


In [8]:
#team6 slicing database to focus on the main usefull content
gpdf02g_1 = gpdf02g.drop(columns=['OBJID', 'ngbhd_code', 'perc_other', 'perc_publcnt', 'perc_trnsn_total', 'perc_publcnt_tree',
                                      'perc_publcnt_green', 'perc_publcnt_greennt', 'perc_built_tree', 'perc_built_green', 'perc_built_greennt',
                                      'perc_other_tree', 'perc_other_green', 'perc_other_greennt', 'perc_trnsn_tree', 'perc_trnsn_green',
                                      'perc_trnsn_greennt', 'perc_water', 'perc_water_tree', 'm2_ngbhd_pls_agrct', 'm2_publicnt_green',
                                      'm2_publicnt', 'm2_publcnt_greennt', 'm2_publicnt_green', 'm2_publicnt', 'm2_publcnt_greennt',
                                      'm2_built_envmt_green', 'm2_built_envmt', 'm2_built_envmt_greennt', 'm2_water', 'm2_agrct_green', 'm2_agrct',
                                      'm2_agrct_greennt', 'm2_green_other', 'm2_other_surfc', 'm2_other_greennt', 'm2_green_trnsn', 'm2_trnsn',
                                      'm2_trnsn_greennt', 'm2_tree_publc', 'm2_tree_publcnt', 'm2_tree_built_envmt', 'm2_tree_water', 'm2_agrct_tree',
                                      'm2_tree_other', 'm2_tree_trnsn', 'perc_built_gray', 'mnpty_code', 'mnpty_name', 'qty_inhbt', 'perc_built_area'])
gpdf02g_1.head()

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,-99999999,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F-buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11


In [9]:
#looking for any cells with no values on them
gpdf02g_1[(gpdf02g_1 == '').any(axis=1)]

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree


In [10]:
#team6 replacing all -99999999 values in pop_density with 0 just to interfere less with our analysis
gpdf02g_1['pop_dnsty_km2'] = gpdf02g_1['pop_dnsty_km2'].replace(-99999999, 0)
gpdf02g_1.head(10)

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,0,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F-buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11
5,4578,IJselbuurt West,1,19329,7.15,133355.16,26.85,47469.49,50.09,3845.38,2.88,14.50,32.71,43624.12,26.81,5.42,21.39
6,4787,Kelbergen,1,4980,3.54,218062.29,11.15,107361.42,71.03,14376.39,6.59,21.80,42.64,92985.03,33.53,9.08,24.45
7,4509,Bedrijventerrein Sloterdijk I,3,129,0.71,682174.55,30.53,254587.23,45.45,55362.94,8.12,8.13,29.20,199224.29,17.45,8.58,8.87
8,4390,Sarphatistrook,1,11904,15.20,216363.28,33.15,66800.83,39.32,2647.74,1.22,8.45,29.65,64153.08,14.91,1.82,13.09
9,4391,de Plantage,1,5285,11.54,378109.04,17.37,63442.67,25.60,5470.57,1.45,8.82,15.33,57972.09,33.41,3.96,29.45


In [11]:
#team6 preparing the neighborhood names to fit more easily with the other data set
gpdf02g_1['ngbhd_name'] = gpdf02g_1['ngbhd_name'].str.replace("-", " ", regex=False)
gpdf02g_1.head(10)

,KVID,ngbhd_name,urbtn_indx_km2,pop_dnsty_km2,perc_water_total,m2_ngbhd,perc_built_envmt,m2_publc_space,perc_publc_space,m2_publc_green,perc_public_green,perc_publc_tree,perc_public_greennt,m2_publc_greennt,perc_green,perc_low_green,perc_tree
0,4427,Westergasfabriek,1,0,12.47,255419.18,7.28,99685.39,68.27,44776.51,17.53,29.24,21.50,54908.88,50.53,18.01,32.52
1,4552,Duivelseiland,1,23729,4.97,61646.98,34.69,23080.23,42.51,202.57,0.33,5.07,37.11,22877.66,13.09,1.02,12.07
2,4539,Valeriusbuurt Oost,1,12248,4.55,120651.50,32.67,38367.50,40.32,2521.47,2.09,8.52,29.71,35846.03,22.33,3.60,18.73
3,4579,IJselbuurt Oost,1,21880,16.80,159417.36,25.28,54516.32,40.28,2078.39,1.30,6.09,32.89,52437.93,17.14,4.09,13.05
4,4774,F buurt,1,20169,6.38,233166.41,24.11,121287.55,56.32,12907.19,5.54,4.30,46.48,108380.36,11.07,5.96,5.11
5,4578,IJselbuurt West,1,19329,7.15,133355.16,26.85,47469.49,50.09,3845.38,2.88,14.50,32.71,43624.12,26.81,5.42,21.39
6,4787,Kelbergen,1,4980,3.54,218062.29,11.15,107361.42,71.03,14376.39,6.59,21.80,42.64,92985.03,33.53,9.08,24.45
7,4509,Bedrijventerrein Sloterdijk I,3,129,0.71,682174.55,30.53,254587.23,45.45,55362.94,8.12,8.13,29.20,199224.29,17.45,8.58,8.87
8,4390,Sarphatistrook,1,11904,15.20,216363.28,33.15,66800.83,39.32,2647.74,1.22,8.45,29.65,64153.08,14.91,1.82,13.09
9,4391,de Plantage,1,5285,11.54,378109.04,17.37,63442.67,25.60,5470.57,1.45,8.82,15.33,57972.09,33.41,3.96,29.45


In [12]:
#team6 needs to look for matches to ease the process in merging both data bases
matches = set(gpdf01_2['BRT_name']) & set(gpdf02g_1['ngbhd_name'])
num_matches = len(matches)
#to count the number of non-matching values:
num_nomatch = len(set(gpdf01_2['BRT_name']) | set(gpdf02g_1['ngbhd_name'])) - num_matches

print(f'Number of matching values: {num_matches}')
print(f'Number of non-matching values: {num_nomatch}')

Number of matching values: 407
Number of non-matching values: 183
